# Discord Clone Tutorial Part 1

# Tutorial part 1: 
* Setting up Convex
* Setting up Clerk
* Setting Friends dashboard

# Getting starter
Install packages and dependencies
```bash
pnpm convex dev
pnpm add -D npm-run-all
pnpm add @clerk/nextjs
pnpm add svix
```



# Build Basic Static UI for messaging.

<img src="public/image/ui.png" alt="ui.png" width="200"/>



**STEPS**
* Import `useState` from React and define a `Message` interface with `sender` and `content` as string properties.
* Set up messages state using useState, initializing it with sample messages, and create input state as an empty string.
* Create a handleSubmit function to prevent default form behavior, add a new message using the input value, and reset input.
* Render Messages and Input Form:
  * Map over messages to display each message.
  * Add a form with an input field bound to input, an onChange event to update input, and a submit button labeled “Send.”

---
**CODE**: _src/app/page.tsx_

```typescript
import { useState } from "react";

interface Message {
    sender: string;
    content: string;
}

export default function Home() {
  const [messages, setMessages] = useState<Message[]>([
      { sender: 'Alice', content: 'Hello World'},
      { sender: 'Bob', content: "hi, alice!'},
  ]);
  const [input, setInput] = useState("");
    
  const handleSubmit = (event: React.FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    createMessage({ sender: 'Alice', content: input})
    setInput("");
  }
    
  return (
    <>
    <div>
      {messages?.map( (message, index) => (
      <div key={index}>
        <strong>{message.sender}</strong>: {message.content}
      </div>
      ))}
      <form onSubmit={handleSubmit}>
        <input 
          type='text' 
          name='message' 
          id='message' 
          value={input} 
          onChange={e => setInput(e.target.value) }
        />
        <button type='submit'>Send</button>
      </form>

    </div>
    </>
  )
```



# Add Convex to make our static UI connect to a database

<img src=attachment:538ef21e-981e-4ee8-918f-9c9beba1d5f3.png alt="convex-dashboard.png" width="500"/>

# Create a DB Schema

**STEPS**
* To set up a schema for user and message data, start by importing defineSchema and defineTable from "convex/server", and the v validation utility from "convex/values".
* Use defineSchema to define tables for messages. Define fields sender and content, both also validated as strings. 

---
**CODE**: _convex/schema.ts_

```typescript
import { defineSchema, defineTable } from "convex/server";
import { v } from "convex/values";

export default defineSchema({
    messages: defineTable({
        sender: v.string(),
        content: v.string(),
    })
});
```

# Define functions to interact with messages table through Convex
**SOURCE**: [Convex queries](https://docs.convex.dev/functions/query-functions)  |  [Convex mutation](https://docs.convex.dev/functions/mutation-functions)

**Queries**: fetch data from the database, check authentication or peform other business logic, and return data back to the client application. The same rules apply to mutations and actions, while HTTP actions use a different routing approach.

**Mutations**: Mutations insert, update and remove data from the database, check authentication or perform other business logic, and optionally return a response to the client application.

**Query/Mutations Constructor**: To actually declare a `query` in Convex you use the query constructor function. Pass it an object with a `handler` function, which returns the query result.

**Query/Mutations Arguments**: Queries accept named arguments. The argument values are accessible as fields of the second parameter of the handler function. Arguments and responses are automatically serialized and deserialized, and you can pass and return most value-like JavaScript data to and from your query. To both declare the types of arguments and to validate them, add an args object using v validators.

**Query Context**: The query constructor enables fetching data, and other Convex features by passing a QueryCtx object to the handler function as the first parameter. Which part of the query context is used depends on what your query needs to do:
* Fetch from the database use the `db` field. Note that we make the handler function an `async` function so we can `await` the promise returned by `db.get()`:
* Return URLs to stored files use the storage field.
* Check user authentication use the auth field.



**Mutation Constructor**: To declare a mutation in Convex use the mutation constructor function. Pass it an object with a handler function, which performs the mutation

---
**CODE**: _convex/functions/message.ts_

```typescript
import { mutation, query } from "../_generated/server";
import { v } from 'convex/values';

export const list = query({
    handler: async (ctx) => {
        return await ctx.db.query("messages").collect();
    },
});

export const create = mutation({
    args: {
        sender: v.string(),
        content: v.string(),
    },
    handler: async (ctx, { sender, content}) => {
        await ctx.db.insert('messages', { sender, content });
    },
})
```
---
**CODE**: _convex/functions/user.ts_

```typescript
import { 
    internalMutation, 
    MutationCtx, 
    query, 
    QueryCtx } from '../_generated/server';
import { v } from 'convex/values';

export const get = query({
    handler: async (ctx) => {
        return await getCurrentUser(ctx);
    },
})

export const upsert = internalMutation({
    args: { 
        username: v.string(),
        image: v.string(),
        clerkId: v.string(),
    },
    handler: async (ctx, args) => {
        const user = await getUserByClerkid(ctx, args.clerkId)

        // If user is found, update. Otherwise, create new user
        if (user) {
            await ctx.db.patch(user._id, {
                username: args.username,
                image: args.image,
            })
        } else {
            await ctx.db.insert('users', {
                username: args.username,
                image: args.image,
                clerkId: args.clerkId,
            });
        }
    },
});

export const remove = internalMutation({
    args: { clerkId: v.string() },
    handler: async (ctx, args) => {
        const user = await getUserByClerkid(ctx, args.clerkId)

        // If user is found, delete.
        if (user) {
            await ctx.db.delete(user._id);
        }
    },
})

const getCurrentUser = async (ctx: QueryCtx | MutationCtx) => {
    const identity = await ctx.auth.getUserIdentity();
    if (!identity) {
        return null;
    }
    return await getUserByClerkid(ctx, identity.subject)
}

const getUserByClerkid = async (
    ctx: QueryCtx | MutationCtx, 
    clerkId: string
) => {
    return await ctx.db
        .query('users')
        .withIndex('by_clerk_id', (q) => q.eq('clerkId', clerkId))
        .unique();
}
```
---

This code should run both next and convex

**CODE**: package.json
```json
{
    ...
    "scripts": {
        "dev" : "run-p dev:*",
        "dev:convex" : "convex dev",
        "dev:next" : "next dev --turbopack",
    }
    ...
}
```

# Create a client component for the Convex provider
[Convex Quickstart](https://docs.convex.dev/quickstart/nextjs)
Add a new file `ConvexClientProvider.tsx` in the `app/` folder. Include the `"use client";` directive, create a `ConvexReactClient` and a component that wraps its children in a `ConvexProvider`. The `NEXT_PUBLIC_CONVEX_URL` should already added in `.env.local` when we signed up with Convex.

---
**CODE**: _src/app/ConvexClientProvider.tsx_
```typescript
"use client";

import { ConvexReactClient } from "convex/react";
import { ConvexProviderWithClerk } from 'convex/react-clerk' 
import { ReactNode } from "react";
import { useAuth } from '@clerk/nextjs';

const convex = new ConvexReactClient(process.env.NEXT_PUBLIC_CONVEX_URL!);

export function ConvexClientProvider({ 
    children, 
}: { 
    children: ReactNode 
}) {
  return (
    <ConvexProviderWithClerk client={convex} useAuth={useAuth}>
        {children}
    </ConvexProviderWithClerk>
  )
}
```

# Wire up the ConvexClientProvider
In `app/layout.tsx`, wrap the children of the body element with the `ConvexClientProvider`.

---
**CODE**: _src/app/layout/tsx_
```typescript

import type { Metadata } from "next";
import "./globals.css";
import { ConvexClientProvider } from "./ConvexClientProvider";
import {
  ClerkProvider,
} from '@clerk/nextjs'



export const metadata: Metadata = {
  title: "Create Next App",
  description: "Generated by create next app",
};

export default function RootLayout({
  children,
}: Readonly<{
  children: React.ReactNode;
}>) {
  return (
    <ClerkProvider dynamic>
    <html lang="en">
      <body>
        <ConvexClientProvider>{children}</ConvexClientProvider>
      </body>
    </html>
    </ClerkProvider>
  );
}

```

# Display the data in your app
In `app/page.tsx`, use the `useQuery` hook to fetch from your `api.functions.message` API function.

---
**CODE**: _src/app/page.tsx_
```typescript
'use client'

import { useMutation, useQuery } from 'convex/react'; 
import { useState } from "react";
import { api } from "../../convex/_generated/api";

export default function Home() {
  const messages = useQuery(api.functions.message.list)
  const createMessage = useMutation(api.functions.message.create)
  const [input, setInput] = useState("");

  const handleSubmit = (event: React.FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    createMessage({ sender: 'Alice', content: input})
    setInput("");
  }

  return (
    <>
    <div>
      {messages?.map( (message, index) => (
      <div key={index}>
        <strong>{message.sender}</strong>: {message.content}
      </div>
      ))}
      <form onSubmit={handleSubmit}>
        <input 
          type='text' 
          name='message' 
          id='message' 
          value={input} 
          onChange={e => setInput(e.target.value) }
        />
        <button type='submit'>Send</button>
      </form>

    </div>
    </>
  )
}
```

When you execute a mutation, all the queries will update automatically across clients because they are all connected to the convex realtime API.

# Setting up Clerk and Convex
**SOURCE**: 
* [Clerk Quickstart](https://clerk.com/docs/quickstarts/nextjs)
* [Convex Clerk Auth](https://docs.convex.dev/auth/clerk)

## Under the hood
The authentication flow looks like this under the hood:
1. The user clicks a login button
2. The user is redirected to a page where they log in via whatever method you configure in Clerk
3. After a successful login Clerk redirects back to your page, or a different page which you configure via the afterSignIn prop.
4. The ClerkProvider now knows that the user is authenticated.
5. The ConvexProviderWithClerk fetches an auth token from Clerk.
6. The ConvexReactClient passes this token down to your Convex backend to validate
7. Your Convex backend retrieves the public key from Clerk to check that the token's signature is valid.
8. The ConvexReactClient is notified of successful authentication, and ConvexProviderWithClerk now knows that the user is authenticated with Convex. useConvexAuth returns isAuthenticated: true and the Authenticated component renders its children.

ConvexProviderWithClerk takes care of refetching the token when needed to make sure the user stays authenticated with your backend.

**STEPS**
* Go to [Clerk dashboard](https://dashboard.clerk.com/apps/app_2oGaC1K1kHQ72iyoOJ3IRSW8bbY/instances/ins_2oGaC6oUHHOguvE9r3DztKHYCnG/user-authentication/email-phone-username).
* Create the application.
* Install `pnpm add @clerk/nextjs`
* Set your environment variables to your `.env.local`. Retrieve the key from the `Dashboard/configure/API` keys page

<img src="public/images/convex_env.png" alt="convex_env.png" width="500"/>

## Debugging Authentication
If a user goes through the Clerk login flow successfully, and after being redirected back to your page useConvexAuth gives isAuthenticated: false, it's possible that your backend isn't correctly configured.

The auth.config.ts file in your convex/ directory contains a list of configured authentication providers. You must run npx convex dev or npx convex deploy after adding a new provider to sync the configuration to your backend.

# Add clerkMiddleware() to your app
`clerkMiddleware()` grants you access to user authentication state throughout your app, on any route or page. It also allows you to protect specific routes from unauthenticated users. To add `clerkMiddleware()` to your app, follow these steps:

* Create a `middleware.ts` file.
    * If you're using the /src directory, create middleware.ts in the /src directory.
    * If you're not using the /src directory, create middleware.ts in the root directory alongside .env.local.
* In your middleware.ts file, export Clerk's clerkMiddleware() helper:
---
**CODE**: _src/middleware.ts_
```typescript
import { clerkMiddleware } from '@clerk/nextjs/server'

export default clerkMiddleware()

export const config = {
  matcher: [
    // Skip Next.js internals and all static files, unless found in search params
    '/((?!_next|[^?]*\\.(?:html?|css|js(?!on)|jpe?g|webp|png|gif|svg|ttf|woff2?|ico|csv|docx?|xlsx?|zip|webmanifest)).*)',
    // Always run for API routes
    '/(api|trpc)(.*)',
  ],
}
```

# Create a JWT Template
This is how we are going to Convex what the user from Clerk is and for them to be able to connect with each other. In the clerk dashboard, go to configure and scroll down to to Jason Web token template.

<img src="attachment:eaba7702-c49a-4e06-a2ef-b1e667cb9658.png" alt='jwt.png' width='500' />

* In the JWT Templates section of the Clerk dashboard tap on + New template and choose Convex
* Copy the Issuer URL from the Issuer input field.
* Hit Apply Changes.

Note: Do NOT rename the JWT token, it must be called convex.

# Create the auth config
In the convex folder create a new file auth.config.ts with the server-side configuration for validating access tokens.

Paste in the Issuer URL from the JWT template and set applicationID to "convex" (the value of the "aud" Claims field).

---
**CODE**: _convex/auth_config.ts_
```typescript
const clerkConfig = {
  providers: [
    {
      domain: "https://tops-worm-73.clerk.accounts.dev",
      applicationID: "convex",
    },
  ],
};

export default clerkConfig;
```

# Add `<ClerkProvider>` and Clerk components to your app

The `<ClerkProvider>` component wraps your app to provide active session and user context to Clerk's hooks and other components.

Note: If you are getting an error from Clerk about useAuth() called in static mode, wrap this component in `<ClerkProvider dynamic` to make auth data available during server-side rendering.

---
**CODE**: _src/app/layout.tsx_

```typescript
import type { Metadata } from "next";
import "./globals.css";
import { ConvexClientProvider } from "./ConvexClientProvider";
import {
  ClerkProvider,
} from '@clerk/nextjs'



export const metadata: Metadata = {
  title: "Create Next App",
  description: "Generated by create next app",
};

export default function RootLayout({
  children,
}: Readonly<{
  children: React.ReactNode;
}>) {
  return (
    <>
    <html lang="en">
      <body>
        <ClerkProvider dynamic>
          <ConvexClientProvider>{children}</ConvexClientProvider>
        </ClerkProvider>
      </body>
    </html>
    </>
  );
}
```

---
**CODE**: _src/app/ConvexClientProvider.tsx_
```typescript
"use client";

import { ConvexReactClient } from "convex/react";
import { ConvexProviderWithClerk } from 'convex/react-clerk' 
import { ReactNode } from "react";
import { useAuth } from '@clerk/nextjs';

// Define Convex client
const convex = new ConvexReactClient(process.env.NEXT_PUBLIC_CONVEX_URL!);

export function ConvexClientProvider({ 
    children, 
}: { 
    children: ReactNode 
}) {
  return (
    <ConvexProviderWithClerk client={convex} useAuth={useAuth}>
        {children}
    </ConvexProviderWithClerk>
  )
}
```

# Show UI based on authentication state
You can control which UI is shown when the user is signed in or signed out with the provided components from "convex/react" and "@clerk/clerk-react". This will allow both Clerk and Convex to know who is signed in.

To get started create a shell that will let the user sign in and sign out.

Because the Content component is a child of Authenticated, within it and any of its children authentication is guaranteed and Convex queries can require it.

---
**CODE**: _src/app/page.tsx_
```typescript
'use client'

import { Authenticated, Unauthenticated, useMutation, useQuery } from 'convex/react'; 
import { useState } from "react";
import { api } from "../../convex/_generated/api";
import { SignInButton, SignOutButton } from '@clerk/nextjs';

export default function Home() {
  // Retrieve messages
  const messages = useQuery(api.functions.message.list)

  // Create a new message
  const createMessage = useMutation(api.functions.message.create)

  const [input, setInput] = useState("");

  const handleSubmit = (event: React.FormEvent<HTMLFormElement>) => {
    event.preventDefault();
    createMessage({ sender: 'Alice', content: input})
    setInput("");
  }

  return (
    <>
    <Authenticated>
    <div>
      {messages?.map( (message, index) => (
      <div key={index}>
        <strong>{message.sender}</strong>: {message.content}
      </div>
      ))}
      <form onSubmit={handleSubmit}>
        <input 
          type='text' 
          name='message' 
          id='message' 
          value={input} 
          onChange={e => setInput(e.target.value) }
        />
        <button type='submit'>Send</button>
      </form>

    </div>
    <SignOutButton />
    
    </Authenticated>
    <Unauthenticated>
      <SignInButton />
    </Unauthenticated>
    </>
  )
}
```

# Setting up users
Currently, our users are stored in Clerk, but in order for users to add each other as friend join servers and send messages that are actually linked back to that specific user, we'll want that information to be stored in our database as well. In order to do that, we need to update our database schema in order to have a user's table. then set up a web hook which is a function that clerk can call on our Convex deployment that can sync the user infromation from Clerk into our database.

In our database, we need to add `clerkId` which is how we're going to match up users between our database and clerk's database.

We also need to add `.index('by_clerk_id`), ['clerkId'])` on our db table to help our database quickly associate a clerk ID with a specific user in our database rather than scanning the entire table to find that user.

---
**CODE**: _convex/schema.ts_
```typescript
import { defineSchema, defineTable } from "convex/server";
import { v } from "convex/values";

export default defineSchema({
    users: defineTable({
        username: v.string(),
        image: v.string(),
        clerkId: v.string(),
    }).index("by_clerk_id", ['clerkId']),
    messages: defineTable({
        sender: v.string(),
        content: v.string(),
    })
});
```

# Create the endpoint in your application

**SOURCE**: [Clerk Webhooks](https://clerk.com/docs/integrations/webhooks/sync-data)

**HTTP action**: is a public HTTP endpoint that clerk will be able to call.

Create a Route Handler that uses `svix` to verify the webhook signature and that receives the webhook's payload.

For the sake of this guide, you will only log the payload to the console. In a real world application, you would use the payload to trigger some action. For example, you are listening for the user.updated event, so you could perform a database `update` or `upsert` to update the user's details.

Your webhook will need to return either an error code, or a success code, like 200 or 201. If it returns an error code, the webhook will reflect that in the Dashboard log and it will retry. When the webhook returns a success code, there will be no retries and the webhook will show a success status in the Dashboard.

**STEPS**:
* Install `svix` on _Terminal $ `pnpm add svix`_
* Add the webhook url from Convex. Go `settings > url & deploy key`

<img src='attachment:0a162cf8-a977-4d31-bd7a-5f39e8b6d02d.png' alt='convex_url.png' width=500 />

* Go to Clerk dashboard > `configure` > webhooks. Add the Url from convex and concatenate `.../clerk-webhook`
  
  <img src='attachment:0fe70e46-3857-4730-b5ca-9d497f61e280.png' alt='webhook.png' width=500 />
  <img src='attachment:3ce77759-d407-4940-96b9-0843ffd03c45.png' alt='convex_url-2.png' width=500 />

* Subscribe to events. Check `user` to enable any event that involves the user.

<img src='attachment:4c61f959-7e67-4db9-8c96-c47059775e4c.png' alt='convex_url-3.png' width=500 />

* Copy the `Signing Secret` and paste it on the environment variables on Convex. This will verify that the signature is coming from Clerk.

<img src='attachment:339d4926-65b0-43ca-b442-bf869259f330.png' alt='clerk_secret.png' width=500 />

<img src='attachment:0be52173-6b59-4b07-8898-1290cbc921f4.png'
alt='convex_env.png' width=500 />

* In the `https.ts` file, we need to define some internal queries and mutations to update the user  and then we can call them from here. There are special protections around Convex queries and mutations to make sure that they actually run the way we expect them to. If they are interrrupted, our data does not end up in an inconsistent state.

**`internalMutation`**: This can't be called from our frontend but we can call it from this HTTP Action or another function within the Convex API. We create 2 functions:
* upsert
* remove

---
**CODE**: _convex/functions.user.ts_

```typescript
import { 
    internalMutation, 
    MutationCtx, 
    query, 
    QueryCtx } from '../_generated/server';
import { v } from 'convex/values';

export const get = query({
    handler: async (ctx) => {
        return await getCurrentUser(ctx);
    },
})

export const upsert = internalMutation({
    args: { 
        username: v.string(),
        image: v.string(),
        clerkId: v.string(),
    },
    handler: async (ctx, args) => {
        const user = await getUserByClerkid(ctx, args.clerkId)

        // If user is found, update. Otherwise, create new user
        if (user) {
            await ctx.db.patch(user._id, {
                username: args.username,
                image: args.image,
            })
        } else {
            await ctx.db.insert('users', {
                username: args.username,
                image: args.image,
                clerkId: args.clerkId,
            });
        }
    },
});

export const remove = internalMutation({
    args: { clerkId: v.string() },
    handler: async (ctx, args) => {
        const user = await getUserByClerkid(ctx, args.clerkId)

        // If user is found, delete.
        if (user) {
            await ctx.db.delete(user._id);
        }
    },
})

const getCurrentUser = async (ctx: QueryCtx | MutationCtx) => {
    const identity = await ctx.auth.getUserIdentity();
    if (!identity) {
        return null;
    }
    return await getUserByClerkid(ctx, identity.subject)
}

const getUserByClerkid = async (
    ctx: QueryCtx | MutationCtx, 
    clerkId: string
) => {
    return await ctx.db
        .query('users')
        .withIndex('by_clerk_id', (q) => q.eq('clerkId', clerkId))
        .unique();
}
```


---
**CODE**: _convex/http.ts_
```typescript
import { httpRouter } from 'convex/server';
import { httpAction } from './_generated/server';
import { Webhook } from 'svix';
import type { WebhookEvent } from '@clerk/nextjs/server';
import { internal } from './_generated/api';

const http = httpRouter();

http.route({
    method: "POST",
    path: '/clerk-webhook',
    handler: httpAction(async (ctx, req) => {
        const body = await validateRequest(req);
        if (!body) {
            return new Response('Unauthorized', { status: 401 });
        }

        switch (body.type) {
            case 'user.created':
                await ctx.runMutation(internal.functions.user.upsert, { 
                    username: body.data.username!,
                    image: body.data.image_url,
                    clerkId: body.data.id,
                });
                break;
            case 'user.updated':
                await ctx.runMutation(internal.functions.user.upsert, { 
                    username: body.data.username!,
                    image: body.data.image_url,
                    clerkId: body.data.id,
                });
                break;
            case 'user.deleted':
                if (body.data.id) {
                    await ctx.runMutation(internal.functions.user.remove, { 
                        clerkId: body.data.id,
                    });
                }
                break;
            default:
                return new Response('Unrecognized event', { status: 400 });
        }

        return new Response('OK', { status: 200 });
    }),
});

const validateRequest = async (req: Request) => {
    const headers = req.headers;
    const text = await req.text();

    const svix_id = headers.get('svix-id');
    const svix_timestamp = headers.get('svix-timestamp');
    const svix_signature = headers.get('svix-signature');

    try {
        const webhook = new Webhook(process.env.CLERK_WEBHOOK_SECRET!);
        return webhook.verify(text, {
            'svix_id': svix_id!,
            'svix_timestamp': svix_timestamp!,
            'svix_signature': svix_signature!,
        }) as unknown as WebhookEvent;
    } catch (error) {
        console.error('Webhook verification failed:', error);
        return null;
    }
};

export default http;

```


# Setting up Dashboard

`npx shadcn init`

In [ ]:
 /?__clerk_db_jwt=dvb_2oGaZpkiYGuu9SXXk3OuVfwERkN 200 in 39ms
Clerk: Refreshing the session token resulted in an infinite redirect loop. This usually means that your Clerk instance keys do not match - make sure to copy the correct publishable and secret keys from the Clerk dashboard.
 GET /?__clerk_db_jwt=dvb_2oGaZpkiYGuu9SXXk3OuVfwERkN 200 in 39ms
 GET / 200 in 37ms